In [12]:
import pandas as pd
import yfinance as yf
import os
import pytz

# New York Zeitzone
NY_TZ = pytz.timezone("America/New_York")

def calculate_indicators(data):
    """Berechnet eine Vielzahl von Indikatoren."""
    
    # Sicherstellen, dass alle Werte numerisch sind
    cols_to_fix = ['High', 'Low', 'Close', 'Volume']
    for col in cols_to_fix:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Fehlende Werte durch 0 ersetzen
    data.fillna({'High': 0, 'Low': 0, 'Close': 0, 'Volume': 0}, inplace=True)

    # **VWAP Berechnung**
    data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['Typical_Price'] = data['Typical_Price'].fillna(0)
    data['Volume'] = data['Volume'].fillna(0)

    data['Cum_TPxV'] = (data['Typical_Price'] * data['Volume']).cumsum()
    data['Cum_Volume'] = data['Volume'].cumsum()

    # Schutz gegen Division durch Null
    data['VWAP'] = data['Cum_TPxV'] / data['Cum_Volume']
    data.loc[data['Cum_Volume'] == 0, 'VWAP'] = None 

    #  Debug-Ausgabe für VWAP
    print(" VWAP Debug-Daten:")
    print(data[['Typical_Price', 'Volume', 'Cum_TPxV', 'Cum_Volume', 'VWAP']].head(20))
    
    return data

def fetch_and_save_stock_data(ticker, start_date, end_date, interval, save_path):
    """
    Lädt Aktienkursdaten und speichert sie als CSV.

    Args:
        ticker (str): Aktien-Ticker (z.B. 'AAPL').
        start_date (str): Startdatum ('YYYY-MM-DD').
        end_date (str): Enddatum ('YYYY-MM-DD').
        interval (str): Intervall ('1h', '1d', etc.).
        save_path (str): Speicherpfad.
    """
    
    try:
        print(f"Abrufe Daten für {ticker} von {start_date} bis {end_date} mit Intervall {interval}...")

        # Daten abrufen
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

        if data.empty:
            print(f"❌ Keine Daten für {ticker} gefunden.")
            return

        # Index in Spalte 'Date' umwandeln *bevor* Zeitzonenkonvertierung
        data = data.reset_index()

        # Datetime Spalte erstellen und in New York Zeit konvertieren
        data['Datetime'] = pd.to_datetime(data['Date'], utc=True).dt.tz_convert(NY_TZ)

        # Indikatoren berechnen
        data = calculate_indicators(data)

        # Speichern als CSV
        os.makedirs(save_path, exist_ok=True)
        file_name = f"{ticker}.csv"
        file_path = os.path.join(save_path, file_name)

        # Datei überschreiben (keine Erweiterung)
        data.to_csv(file_path, index=False)
        print(f"✅ Datei gespeichert: {file_path}")

    except Exception as e:
        print(f"⚠️ Fehler für {ticker}: {e}")
def main():
    # Aktienliste & Parameter
    stocks = ['AAPL', 'MSFT', 'GOOGL', 'META']
    start_date = '2023-03-01'
    end_date = '2024-12-31'
    interval = '1h'
    save_path = "C:/Users/Martin/Documents/StockData"  # Passe den Pfad an!

    # Daten abrufen & speichern
    for stock in stocks:
        fetch_and_save_stock_data(stock, start_date, end_date, interval, save_path)
        

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Abrufe Daten für AAPL von 2023-03-01 bis 2024-12-31 mit Intervall 1h...
⚠️ Fehler für AAPL: 'Date'
Abrufe Daten für MSFT von 2023-03-01 bis 2024-12-31 mit Intervall 1h...
⚠️ Fehler für MSFT: 'Date'
Abrufe Daten für GOOGL von 2023-03-01 bis 2024-12-31 mit Intervall 1h...



[*********************100%***********************]  1 of 1 completed

⚠️ Fehler für GOOGL: 'Date'
Abrufe Daten für META von 2023-03-01 bis 2024-12-31 mit Intervall 1h...
⚠️ Fehler für META: 'Date'
